In [1]:
import numpy as np
import pandas as pd
import requests
import time
from scipy.stats import linregress
from pprint import pprint
import json
from census import Census
from config import (census_key, gkey)
import gmaps

# Census API Key
c = Census(census_key)

### Census Information API on poverty and race per zipcode

In [2]:
# Run Census Search to retrieve data on all zip codes (latest available Census)
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
census_data = c.acs5.get(("B01003_001E", "B17001_002E", "B19013_001E", "B02001_002E", "B02001_003E", "B02001_004E", "B02001_005E", "B02001_006E", "B02001_008E", "B03001_003E"
                         ), {'for': 'zip code tabulation area:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B17001_002E": "Poverty Count",
                                      "B19013_001E": "Median Income",
                                      "B02001_002E": "Caucasian",
                                      "B02001_003E": "African American",
                                      "B02001_004E": "American Indian",
                                      "B02001_005E": "Asian",
                                      "B02001_006E": "Native Hawaiian",
                                      "B02001_008E": "Two or more races",
                                      "B03001_003E": "Hispanic",
                                      "zip code tabulation area": "Zipcode"})

In [3]:
census_pd.sample(5)

,Population,Poverty Count,Median Income,Caucasian,African American,American Indian,Asian,Native Hawaiian,Two or more races,Hispanic,state,Zipcode
11650,19746.0,2748.0,50809.0,14059.0,4906.0,46.0,296.0,0.0,308.0,555.0,01,36207
6533,32489.0,2391.0,80855.0,4157.0,26472.0,67.0,605.0,10.0,1015.0,921.0,24,21133
5225,1018.0,137.0,48654.0,991.0,0.0,8.0,8.0,0.0,2.0,9.0,42,17006
28274,11799.0,939.0,53707.0,11290.0,1.0,41.0,194.0,0.0,62.0,919.0,08,81506
13520,28667.0,4657.0,39778.0,24097.0,3566.0,93.0,116.0,0.0,689.0,715.0,21,42003


In [4]:
census_pd["Poverty Rate"] = 100 * census_pd["Poverty Count"]/ census_pd["Population"]

In [5]:
census_pd.sample(5)

,Population,Poverty Count,Median Income,Caucasian,African American,American Indian,Asian,Native Hawaiian,Two or more races,Hispanic,state,Zipcode,Poverty Rate
26722,28558.0,4404.0,46801.0,8458.0,13480.0,60.0,1861.0,0.0,289.0,8630.0,48,77071,15.421248
5678,9339.0,1041.0,76413.0,8823.0,208.0,11.0,136.0,0.0,58.0,404.0,42,18222,11.146804
4335,4670.0,515.0,51028.0,4341.0,123.0,8.0,41.0,0.0,82.0,131.0,36,14883,11.027837
23090,550.0,127.0,26667.0,530.0,0.0,0.0,0.0,0.0,20.0,34.0,20,66851,23.090909
3172,4800.0,484.0,59159.0,4500.0,266.0,0.0,0.0,0.0,27.0,28.0,36,12143,10.083333


In [6]:
census_pd.columns = census_pd.columns.str.replace(' ', '_').str.lower()
census_pd.tail(5)

,population,poverty_count,median_income,caucasian,african_american,american_indian,asian,native_hawaiian,two_or_more_races,hispanic,state,zipcode,poverty_rate
33115,15.0,0.0,-666666666.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,02,99923,0.000000
33116,927.0,172.0,57375.0,446.0,3.0,366.0,0.0,7.0,105.0,26.0,02,99925,18.554477
33117,1635.0,235.0,53409.0,239.0,0.0,1210.0,22.0,0.0,147.0,116.0,02,99926,14.373089
33118,38.0,28.0,-666666666.0,38.0,0.0,0.0,0.0,0.0,0.0,0.0,02,99927,73.684211
33119,2484.0,192.0,57583.0,1670.0,2.0,496.0,69.0,9.0,208.0,121.0,02,99929,7.729469


In [7]:
census_pd["caucasian_%"] = 100 * census_pd["caucasian"]/ census_pd["population"]
census_pd["african_american_%"] = 100 * census_pd["african_american"]/ census_pd["population"]
census_pd["american_indian_%"] = 100 * census_pd["american_indian"]/ census_pd["population"]
census_pd["asian_%"] = 100 * census_pd["asian"]/ census_pd["population"]
census_pd["native_hawaiian_%"] = 100 * census_pd["native_hawaiian"]/ census_pd["population"]
census_pd["two_or_more_races_%"] = 100 * census_pd["two_or_more_races"]/ census_pd["population"]
census_pd["hispanic_%"] = 100 * census_pd["hispanic"]/ census_pd["population"]
clean_census_df=census_pd[["zipcode", "population", "poverty_rate", "median_income", "caucasian_%", "african_american_%", "american_indian_%", "asian_%", "native_hawaiian_%", "two_or_more_races_%", "hispanic_%"]]

In [8]:
clean_census_df = clean_census_df.rename(columns={'zipcode': 'zip_code'})
clean_census_df['zip_code'].apply(type).value_counts()

<class 'str'>    33120
Name: zip_code, dtype: int64

In [9]:
clean_census_df

,zip_code,population,poverty_rate,median_income,caucasian_%,african_american_%,american_indian_%,asian_%,native_hawaiian_%,two_or_more_races_%,hispanic_%
0,00601,17242.0,62.475351,13092.0,75.548080,0.840970,0.144995,0.017399,0.000000,0.661176,99.663612
1,00602,38442.0,51.014515,16358.0,79.415743,2.783414,0.000000,0.000000,0.000000,13.794808,93.452474
2,00603,48814.0,49.856599,16603.0,76.473962,3.953784,0.235588,0.745688,0.020486,2.167411,97.412628
3,00606,6437.0,64.672984,12832.0,40.810937,2.314743,0.279633,0.000000,0.000000,1.941898,99.844648
4,00610,27073.0,43.305138,19309.0,75.540206,2.570827,0.000000,0.000000,0.003694,11.280612,96.195472
...,...,...,...,...,...,...,...,...,...,...,...
33115,99923,15.0,0.000000,-666666666.0,100.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
33116,99925,927.0,18.554477,57375.0,48.112190,0.323625,39.482201,0.000000,0.755124,11.326861,2.804746
33117,99926,1635.0,14.373089,53409.0,14.617737,0.000000,74.006116,1.345566,0.000000,8.990826,7.094801
33118,99927,38.0,73.684211,-666666666.0,100.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [10]:
clean_census_df['median_income'].apply(type).value_counts()

<class 'float'>    33120
Name: median_income, dtype: int64

In [11]:
(clean_census_df.median_income <= 0).value_counts()

False    30926
True      2194
Name: median_income, dtype: int64

In [12]:
updated_census_df = clean_census_df[(clean_census_df['median_income']>0)]
updated_census_df

,zip_code,population,poverty_rate,median_income,caucasian_%,african_american_%,american_indian_%,asian_%,native_hawaiian_%,two_or_more_races_%,hispanic_%
0,00601,17242.0,62.475351,13092.0,75.548080,0.840970,0.144995,0.017399,0.000000,0.661176,99.663612
1,00602,38442.0,51.014515,16358.0,79.415743,2.783414,0.000000,0.000000,0.000000,13.794808,93.452474
2,00603,48814.0,49.856599,16603.0,76.473962,3.953784,0.235588,0.745688,0.020486,2.167411,97.412628
3,00606,6437.0,64.672984,12832.0,40.810937,2.314743,0.279633,0.000000,0.000000,1.941898,99.844648
4,00610,27073.0,43.305138,19309.0,75.540206,2.570827,0.000000,0.000000,0.003694,11.280612,96.195472
...,...,...,...,...,...,...,...,...,...,...,...
33113,99921,1986.0,14.400806,58571.0,68.026183,0.553877,15.508560,1.963746,0.000000,13.595166,4.028197
33114,99922,330.0,39.090909,34028.0,12.424242,1.212121,71.818182,0.000000,8.787879,5.757576,0.000000
33116,99925,927.0,18.554477,57375.0,48.112190,0.323625,39.482201,0.000000,0.755124,11.326861,2.804746
33117,99926,1635.0,14.373089,53409.0,14.617737,0.000000,74.006116,1.345566,0.000000,8.990826,7.094801


In [13]:
final_census_df=updated_census_df[["zip_code", "population", "poverty_rate", "median_income"]]
final_census_df

,zip_code,population,poverty_rate,median_income
0,00601,17242.0,62.475351,13092.0
1,00602,38442.0,51.014515,16358.0
2,00603,48814.0,49.856599,16603.0
3,00606,6437.0,64.672984,12832.0
4,00610,27073.0,43.305138,19309.0
...,...,...,...,...
33113,99921,1986.0,14.400806,58571.0
33114,99922,330.0,39.090909,34028.0
33116,99925,927.0,18.554477,57375.0
33117,99926,1635.0,14.373089,53409.0


In [14]:
final_census_df.to_csv("output_data/final_census_df.csv", index=False)

### CSV uploaded on Value of Care

In [15]:
value_df = pd.read_csv("data/Payment_and_Value_of_Care-Hospital.csv", low_memory=False, dtype=str)
value_df.sample(10)

,Facility ID,Facility Name,Address,City,State,ZIP Code,County Name,Phone Number,Payment Measure ID,Payment Measure Name,...,Payment,Lower Estimate,Higher Estimate,Payment Footnote,Value of Care Display ID,Value of Care Display Name,Value of Care Category,Value of Care Footnote,Start Date,End Date
8199,230121,MEMORIAL HEALTHCARE,826 WEST KING STREET,OWOSSO,MI,48867,SHIAWASSEE,(989) 723-5211,PAYM_90_HIP_KNEE,Payment for hip/knee replacement patients,...,"$18,844","$17,604","$20,131",NaN,COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Average Complications and Lower Payment,NaN,04/01/2016,03/31/2019
5763,161307,HANCOCK COUNTY HEALTH SYSTEM,532 1ST ST NW,BRITT,IA,50423,HANCOCK,(641) 843-5000,PAYM_90_HIP_KNEE,Payment for hip/knee replacement patients,...,Not Available,Not Available,Not Available,5,COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Not Available,13,04/01/2016,03/31/2019
9778,261329,COX MONETT HOSPITAL,801 NORTH LINCOLN AVENUE,MONETT,MO,65708,BARRY,(417) 354-1400,PAYM_30_PN,Payment for pneumonia patients,...,"$16,263","$13,694","$18,764",NaN,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Average Payment,NaN,07/01/2016,06/30/2019
15296,440104,ERLANGER MEDICAL CENTER,975 E 3RD ST,CHATTANOOGA,TN,37403,HAMILTON,(423) 778-7000,PAYM_30_AMI,Payment for heart attack patients,...,"$26,207","$24,478","$28,044",NaN,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,NaN,07/01/2016,06/30/2019
10089,280013,THE NEBRASKA MEDICAL CENTER,987400 NEBRASKA MEDICAL CENTER,OMAHA,NE,68198,DOUGLAS,(402) 552-2040,PAYM_30_HF,Payment for heart failure patients,...,"$17,935","$16,927","$19,056",NaN,MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Average Payment,NaN,07/01/2016,06/30/2019
13579,380050,SKY LAKES MEDICAL CENTER,2865 DAGGETT AVENUE,KLAMATH FALLS,OR,97601,KLAMATH,(541) 274-6150,PAYM_90_HIP_KNEE,Payment for hip/knee replacement patients,...,"$18,952","$18,270","$19,658",NaN,COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Average Complications and Lower Payment,NaN,04/01/2016,03/31/2019
15227,440053,SAINT THOMAS RUTHERFORD HOSPITAL,1700 MEDICAL CENTER PARKWAY,MURFREESBORO,TN,37129,RUTHERFORD,(615) 396-4100,PAYM_90_HIP_KNEE,Payment for hip/knee replacement patients,...,"$17,164","$16,747","$17,596",NaN,COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Average Complications and Lower Payment,NaN,04/01/2016,03/31/2019
6946,181329,SAINT JOSEPH BEREA,305 ESTILL STREET,BEREA,KY,40403,MADISON,(859) 986-6500,PAYM_30_PN,Payment for pneumonia patients,...,"$18,847","$16,797","$20,933",NaN,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Average Payment,NaN,07/01/2016,06/30/2019
8439,231319,ASPIRUS KEWEENAW HOSPITAL AND CLINICS,205 OSCEOLA,LAURIUM,MI,49913,HOUGHTON,(906) 337-6500,PAYM_90_HIP_KNEE,Payment for hip/knee replacement patients,...,"$20,429","$18,838","$22,079",NaN,COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Average Complications and Average Payment,NaN,04/01/2016,03/31/2019
8874,241345,MAYO CLINIC HEALTH SYSTEM - WASECA,501 NORTH STATE STREET,WASECA,MN,56093,WASECA,(507) 835-1210,PAYM_30_PN,Payment for pneumonia patients,...,Not Available,Not Available,Not Available,1,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Not Available,13,07/01/2016,06/30/2019


In [16]:
only_nj_zips=value_df[value_df["State"] == "NJ"]
only_nj_zips

,Facility ID,Facility Name,Address,City,State,ZIP Code,County Name,Phone Number,Payment Measure ID,Payment Measure Name,...,Payment,Lower Estimate,Higher Estimate,Payment Footnote,Value of Care Display ID,Value of Care Display Name,Value of Care Category,Value of Care Footnote,Start Date,End Date
10688,310001,HACKENSACK UNIVERSITY MEDICAL CENTER,30 PROSPECT AVE,HACKENSACK,NJ,07601,BERGEN,(551) 996-2000,PAYM_30_AMI,Payment for heart attack patients,...,"$28,471","$26,974","$30,113",NaN,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Better Mortality and Higher Payment,NaN,07/01/2016,06/30/2019
10689,310001,HACKENSACK UNIVERSITY MEDICAL CENTER,30 PROSPECT AVE,HACKENSACK,NJ,07601,BERGEN,(551) 996-2000,PAYM_30_HF,Payment for heart failure patients,...,"$22,201","$21,378","$23,089",NaN,MORT_PAYM_30_HF,Value of Care Heart Failure measur,Better Mortality and Higher Payment,NaN,07/01/2016,06/30/2019
10690,310001,HACKENSACK UNIVERSITY MEDICAL CENTER,30 PROSPECT AVE,HACKENSACK,NJ,07601,BERGEN,(551) 996-2000,PAYM_30_PN,Payment for pneumonia patients,...,"$21,324","$20,648","$22,048",NaN,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Better Mortality and Higher Payment,NaN,07/01/2016,06/30/2019
10691,310001,HACKENSACK UNIVERSITY MEDICAL CENTER,30 PROSPECT AVE,HACKENSACK,NJ,07601,BERGEN,(551) 996-2000,PAYM_90_HIP_KNEE,Payment for hip/knee replacement patients,...,"$20,870","$20,501","$21,259",NaN,COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Average Complications and Average Payment,NaN,04/01/2016,03/31/2019
10692,310002,NEWARK BETH ISRAEL MEDICAL CENTER,201 LYONS AVE,NEWARK,NJ,07112,ESSEX,(973) 926-7850,PAYM_30_AMI,Payment for heart attack patients,...,"$26,155","$23,703","$28,847",NaN,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,NaN,07/01/2016,06/30/2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10943,313300,CHILDREN'S SPECIALIZED HOSPITAL,200 SOMERSET STREET,NEW BRUNSWICK,NJ,08901,MIDDLESEX,(732) 258-7050,PAYM_90_HIP_KNEE,Payment for hip/knee replacement patients,...,Not Available,Not Available,Not Available,19,COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Not Available,19,04/01/2016,03/31/2019
10944,313302,WEISMAN CHILDRENS REHABILITATION HOSPITAL,"92 BRICK ROAD, 3RD FLOOR",MARLTON,NJ,08053,BURLINGTON,(856) 489-4520,PAYM_30_AMI,Payment for heart attack patients,...,Not Available,Not Available,Not Available,19,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Not Available,19,07/01/2016,06/30/2019
10945,313302,WEISMAN CHILDRENS REHABILITATION HOSPITAL,"92 BRICK ROAD, 3RD FLOOR",MARLTON,NJ,08053,BURLINGTON,(856) 489-4520,PAYM_30_HF,Payment for heart failure patients,...,Not Available,Not Available,Not Available,19,MORT_PAYM_30_HF,Value of Care Heart Failure measur,Not Available,19,07/01/2016,06/30/2019
10946,313302,WEISMAN CHILDRENS REHABILITATION HOSPITAL,"92 BRICK ROAD, 3RD FLOOR",MARLTON,NJ,08053,BURLINGTON,(856) 489-4520,PAYM_30_PN,Payment for pneumonia patients,...,Not Available,Not Available,Not Available,19,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Not Available,19,07/01/2016,06/30/2019


In [17]:
value_df.columns = value_df.columns.str.replace(' ', '_').str.lower()
value_df.head()

,facility_id,facility_name,address,city,state,zip_code,county_name,phone_number,payment_measure_id,payment_measure_name,...,payment,lower_estimate,higher_estimate,payment_footnote,value_of_care_display_id,value_of_care_display_name,value_of_care_category,value_of_care_footnote,start_date,end_date
0,010001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_30_AMI,Payment for heart attack patients,...,"$24,934","$23,490","$26,429",NaN,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,NaN,07/01/2016,06/30/2019
1,010001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_30_HF,Payment for heart failure patients,...,"$17,854","$17,061","$18,676",NaN,MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Average Payment,NaN,07/01/2016,06/30/2019
2,010001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_30_PN,Payment for pneumonia patients,...,"$20,216","$19,186","$21,271",NaN,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Higher Payment,NaN,07/01/2016,06/30/2019
3,010001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_90_HIP_KNEE,Payment for hip/knee replacement patients,...,"$22,216","$21,191","$23,313",NaN,COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Average Complications and Higher Payment,NaN,04/01/2016,03/31/2019
4,010005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,MARSHALL,(256) 593-8310,PAYM_30_AMI,Payment for heart attack patients,...,"$24,742","$22,016","$27,967",NaN,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,NaN,07/01/2016,06/30/2019


In [18]:
for column in value_df.columns:
    print(column)

facility_id
facility_name
address
city
state
zip_code
county_name
phone_number
payment_measure_id
payment_measure_name
payment_category
denominator
payment
lower_estimate
higher_estimate
payment_footnote
value_of_care_display_id
value_of_care_display_name
value_of_care_category
value_of_care_footnote
start_date
end_date


In [19]:
updated_df=value_df.drop(['county_name', 'phone_number', 'payment_footnote', 'value_of_care_footnote', 'lower_estimate', 'higher_estimate', 'start_date', 'end_date', 'payment_measure_id'], axis=1).reset_index(drop=True)
updated_df

,facility_id,facility_name,address,city,state,zip_code,payment_measure_name,payment_category,denominator,payment,value_of_care_display_id,value_of_care_display_name,value_of_care_category
0,010001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for heart attack patients,No Different Than the National Average Payment,574,"$24,934",MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment
1,010001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for heart failure patients,No Different Than the National Average Payment,823,"$17,854",MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Average Payment
2,010001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for pneumonia patients,Greater Than the National Average Payment,536,"$20,216",MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Higher Payment
3,010001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for hip/knee replacement patients,Greater Than the National Average Payment,245,"$22,216",COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Average Complications and Higher Payment
4,010005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,Payment for heart attack patients,No Different Than the National Average Payment,71,"$24,742",MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18835,670136,BAYLOR SCOTT & WHITE MEDICAL CENTER- AUSTIN,5245 W US 290,AUSTIN,TX,78735,Payment for hip/knee replacement patients,Not Available,Not Available,Not Available,COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Not Available
18836,670143,ASCENSION SETON BASTROP,"630 HIGWAY 71 W,",BASTROP,TX,78602,Payment for heart attack patients,Not Available,Not Available,Not Available,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Not Available
18837,670143,ASCENSION SETON BASTROP,"630 HIGWAY 71 W,",BASTROP,TX,78602,Payment for heart failure patients,Not Available,Not Available,Not Available,MORT_PAYM_30_HF,Value of Care Heart Failure measur,Not Available
18838,670143,ASCENSION SETON BASTROP,"630 HIGWAY 71 W,",BASTROP,TX,78602,Payment for pneumonia patients,Not Available,Not Available,Not Available,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Not Available


In [20]:
no_nans_df = updated_df[updated_df.payment != 'Not Available']
no_nans_df

,facility_id,facility_name,address,city,state,zip_code,payment_measure_name,payment_category,denominator,payment,value_of_care_display_id,value_of_care_display_name,value_of_care_category
0,010001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for heart attack patients,No Different Than the National Average Payment,574,"$24,934",MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment
1,010001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for heart failure patients,No Different Than the National Average Payment,823,"$17,854",MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Average Payment
2,010001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for pneumonia patients,Greater Than the National Average Payment,536,"$20,216",MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Higher Payment
3,010001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for hip/knee replacement patients,Greater Than the National Average Payment,245,"$22,216",COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Average Complications and Higher Payment
4,010005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,Payment for heart attack patients,No Different Than the National Average Payment,71,"$24,742",MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18786,670120,THE HOSPITALS OF PROVIDENCE TRANSMOUNTAIN CAMPUS,2000 TRANSMOUNTAIN RD,EL PASO,TX,79911,Payment for pneumonia patients,No Different Than the National Average Payment,143,"$20,221",MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Average Payment
18792,670122,HOUSTON METHODIST THE WOODLANDS HOSPITAL,17201 INTERSTATE 45 SOUTH,THE WOODLANDS,TX,77385,Payment for heart attack patients,No Different Than the National Average Payment,53,"$27,138",MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment
18793,670122,HOUSTON METHODIST THE WOODLANDS HOSPITAL,17201 INTERSTATE 45 SOUTH,THE WOODLANDS,TX,77385,Payment for heart failure patients,Greater Than the National Average Payment,155,"$21,647",MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Higher Payment
18794,670122,HOUSTON METHODIST THE WOODLANDS HOSPITAL,17201 INTERSTATE 45 SOUTH,THE WOODLANDS,TX,77385,Payment for pneumonia patients,No Different Than the National Average Payment,205,"$19,409",MORT_PAYM_30_PN,Value of Care Pneumonia measure,Better Mortality and Average Payment


In [21]:
no_nans_df.state.value_counts()

CA    912
TX    891
FL    625
IL    533
PA    500
NY    488
OH    483
MI    375
IN    338
GA    333
NC    333
WI    329
MO    295
VA    271
IA    259
TN    258
KY    248
KS    244
LA    241
OK    239
NJ    237
WA    230
AL    226
MA    214
MN    208
AZ    203
CO    200
MS    198
AR    189
SC    170
OR    170
MD    167
NE    165
WV    121
UT    110
ME    108
CT     99
NV     98
NM     93
MT     88
ID     88
NH     87
SD     82
ND     62
PR     59
WY     57
VT     46
HI     43
AK     36
RI     35
DE     24
DC     21
VI      6
MP      3
GU      3
Name: state, dtype: int64

In [22]:
no_nans_df.payment_measure_name.unique()

array(['Payment for heart attack patients',
       'Payment for heart failure patients',
       'Payment for pneumonia patients',
       'Payment for hip/knee replacement patients'], dtype=object)

In [23]:
no_nans_df.payment_category.unique()

array(['No Different Than the National Average Payment',
       'Greater Than the National Average Payment',
       'Less Than the National Average Payment'], dtype=object)

In [24]:
no_nans_df.value_of_care_category.unique()

array(['Average Mortality and Average Payment',
       'Average Mortality and Higher Payment',
       'Average Complications and Higher Payment',
       'Worse Mortality and Average Payment',
       'Worse Mortality and Lower Payment',
       'Average Complications and Average Payment',
       'Average Mortality and Lower Payment',
       'Average Complications and Lower Payment',
       'Better Mortality and Average Payment',
       'Worse Complications and Higher Payment',
       'Better Mortality and Lower Payment',
       'Worse Mortality and Higher Payment',
       'Worse Complications and Lower Payment',
       'Better Mortality and Higher Payment',
       'Better Complications and Lower Payment',
       'Better Complications and Average Payment',
       'Better Complications and Higher Payment',
       'Worse Complications and Average Payment', 'Not Available'],
      dtype=object)

In [25]:
clean_value_df=no_nans_df[no_nans_df["value_of_care_category"] != "Not Available"]
clean_value_df

,facility_id,facility_name,address,city,state,zip_code,payment_measure_name,payment_category,denominator,payment,value_of_care_display_id,value_of_care_display_name,value_of_care_category
0,010001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for heart attack patients,No Different Than the National Average Payment,574,"$24,934",MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment
1,010001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for heart failure patients,No Different Than the National Average Payment,823,"$17,854",MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Average Payment
2,010001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for pneumonia patients,Greater Than the National Average Payment,536,"$20,216",MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Higher Payment
3,010001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for hip/knee replacement patients,Greater Than the National Average Payment,245,"$22,216",COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Average Complications and Higher Payment
4,010005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,Payment for heart attack patients,No Different Than the National Average Payment,71,"$24,742",MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18786,670120,THE HOSPITALS OF PROVIDENCE TRANSMOUNTAIN CAMPUS,2000 TRANSMOUNTAIN RD,EL PASO,TX,79911,Payment for pneumonia patients,No Different Than the National Average Payment,143,"$20,221",MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Average Payment
18792,670122,HOUSTON METHODIST THE WOODLANDS HOSPITAL,17201 INTERSTATE 45 SOUTH,THE WOODLANDS,TX,77385,Payment for heart attack patients,No Different Than the National Average Payment,53,"$27,138",MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment
18793,670122,HOUSTON METHODIST THE WOODLANDS HOSPITAL,17201 INTERSTATE 45 SOUTH,THE WOODLANDS,TX,77385,Payment for heart failure patients,Greater Than the National Average Payment,155,"$21,647",MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Higher Payment
18794,670122,HOUSTON METHODIST THE WOODLANDS HOSPITAL,17201 INTERSTATE 45 SOUTH,THE WOODLANDS,TX,77385,Payment for pneumonia patients,No Different Than the National Average Payment,205,"$19,409",MORT_PAYM_30_PN,Value of Care Pneumonia measure,Better Mortality and Average Payment


In [26]:
clean_value_df[clean_value_df.columns[9]] = clean_value_df[clean_value_df.columns[9]].replace('[\$,]', '', regex=True).astype(float)
clean_value_df

<ipython-input-26-d458df61f040>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_value_df[clean_value_df.columns[9]] = clean_value_df[clean_value_df.columns[9]].replace('[\$,]', '', regex=True).astype(float)


,facility_id,facility_name,address,city,state,zip_code,payment_measure_name,payment_category,denominator,payment,value_of_care_display_id,value_of_care_display_name,value_of_care_category
0,010001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for heart attack patients,No Different Than the National Average Payment,574,24934.0,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment
1,010001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for heart failure patients,No Different Than the National Average Payment,823,17854.0,MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Average Payment
2,010001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for pneumonia patients,Greater Than the National Average Payment,536,20216.0,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Higher Payment
3,010001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for hip/knee replacement patients,Greater Than the National Average Payment,245,22216.0,COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Average Complications and Higher Payment
4,010005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,Payment for heart attack patients,No Different Than the National Average Payment,71,24742.0,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18786,670120,THE HOSPITALS OF PROVIDENCE TRANSMOUNTAIN CAMPUS,2000 TRANSMOUNTAIN RD,EL PASO,TX,79911,Payment for pneumonia patients,No Different Than the National Average Payment,143,20221.0,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Average Payment
18792,670122,HOUSTON METHODIST THE WOODLANDS HOSPITAL,17201 INTERSTATE 45 SOUTH,THE WOODLANDS,TX,77385,Payment for heart attack patients,No Different Than the National Average Payment,53,27138.0,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment
18793,670122,HOUSTON METHODIST THE WOODLANDS HOSPITAL,17201 INTERSTATE 45 SOUTH,THE WOODLANDS,TX,77385,Payment for heart failure patients,Greater Than the National Average Payment,155,21647.0,MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Higher Payment
18794,670122,HOUSTON METHODIST THE WOODLANDS HOSPITAL,17201 INTERSTATE 45 SOUTH,THE WOODLANDS,TX,77385,Payment for pneumonia patients,No Different Than the National Average Payment,205,19409.0,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Better Mortality and Average Payment


In [27]:
clean_value_df['payment'].apply(type).value_counts()

<class 'float'>    12135
Name: payment, dtype: int64

In [28]:
clean_value_df['zip_code'].apply(type).value_counts()

<class 'str'>    12135
Name: zip_code, dtype: int64

In [29]:
clean_value_df['denominator'] = clean_value_df['denominator'].apply(lambda x: x.replace('$', '').replace(',', '')).astype(float)
clean_value_df.sample(5)

<ipython-input-29-4183c62c1256>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_value_df['denominator'] = clean_value_df['denominator'].apply(lambda x: x.replace('$', '').replace(',', '')).astype(float)


,facility_id,facility_name,address,city,state,zip_code,payment_measure_name,payment_category,denominator,payment,value_of_care_display_id,value_of_care_display_name,value_of_care_category
4508,140052,OSF SAINT ANTHONY'S HEALTH CENTER,ST ANTHONY'S WAY,ALTON,IL,62002,Payment for heart attack patients,No Different Than the National Average Payment,43.0,25013.0,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment
5332,150100,ASCENSION ST VINCENT EVANSVILLE,3700 WASHINGTON AVE,EVANSVILLE,IN,47750,Payment for heart attack patients,No Different Than the National Average Payment,318.0,26801.0,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment
12608,360123,UH ST JOHN MEDICAL CENTER,29000 CENTER RIDGE ROAD,WESTLAKE,OH,44145,Payment for heart attack patients,No Different Than the National Average Payment,166.0,25360.0,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment
17053,471301,GIFFORD MEDICAL CENTER,44 SOUTH MAIN STREET,RANDOLPH,VT,05060,Payment for heart failure patients,Less Than the National Average Payment,111.0,15585.0,MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Lower Payment
6745,180079,HARRISON MEMORIAL HOSPITAL,1210 KY HWY 36 E,CYNTHIANA,KY,41031,Payment for heart failure patients,No Different Than the National Average Payment,36.0,17514.0,MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Average Payment


In [30]:
clean_value_df['outcome'] = clean_value_df['value_of_care_display_id'].map(lambda v: v.split('_')[0])
clean_value_df

<ipython-input-30-317e12ed0950>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_value_df['outcome'] = clean_value_df['value_of_care_display_id'].map(lambda v: v.split('_')[0])


,facility_id,facility_name,address,city,state,zip_code,payment_measure_name,payment_category,denominator,payment,value_of_care_display_id,value_of_care_display_name,value_of_care_category,outcome
0,010001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for heart attack patients,No Different Than the National Average Payment,574.0,24934.0,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,MORT
1,010001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for heart failure patients,No Different Than the National Average Payment,823.0,17854.0,MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Average Payment,MORT
2,010001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for pneumonia patients,Greater Than the National Average Payment,536.0,20216.0,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Higher Payment,MORT
3,010001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for hip/knee replacement patients,Greater Than the National Average Payment,245.0,22216.0,COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Average Complications and Higher Payment,COMP
4,010005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,Payment for heart attack patients,No Different Than the National Average Payment,71.0,24742.0,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,MORT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18786,670120,THE HOSPITALS OF PROVIDENCE TRANSMOUNTAIN CAMPUS,2000 TRANSMOUNTAIN RD,EL PASO,TX,79911,Payment for pneumonia patients,No Different Than the National Average Payment,143.0,20221.0,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Average Payment,MORT
18792,670122,HOUSTON METHODIST THE WOODLANDS HOSPITAL,17201 INTERSTATE 45 SOUTH,THE WOODLANDS,TX,77385,Payment for heart attack patients,No Different Than the National Average Payment,53.0,27138.0,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,MORT
18793,670122,HOUSTON METHODIST THE WOODLANDS HOSPITAL,17201 INTERSTATE 45 SOUTH,THE WOODLANDS,TX,77385,Payment for heart failure patients,Greater Than the National Average Payment,155.0,21647.0,MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Higher Payment,MORT
18794,670122,HOUSTON METHODIST THE WOODLANDS HOSPITAL,17201 INTERSTATE 45 SOUTH,THE WOODLANDS,TX,77385,Payment for pneumonia patients,No Different Than the National Average Payment,205.0,19409.0,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Better Mortality and Average Payment,MORT


In [31]:
clean_value_df['value_code'] = clean_value_df['value_of_care_display_id'].map(lambda v: v.split('_')[-1])
clean_value_df

<ipython-input-31-f0ca45d6bc9e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_value_df['value_code'] = clean_value_df['value_of_care_display_id'].map(lambda v: v.split('_')[-1])


,facility_id,facility_name,address,city,state,zip_code,payment_measure_name,payment_category,denominator,payment,value_of_care_display_id,value_of_care_display_name,value_of_care_category,outcome,value_code
0,010001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for heart attack patients,No Different Than the National Average Payment,574.0,24934.0,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,MORT,AMI
1,010001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for heart failure patients,No Different Than the National Average Payment,823.0,17854.0,MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Average Payment,MORT,HF
2,010001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for pneumonia patients,Greater Than the National Average Payment,536.0,20216.0,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Higher Payment,MORT,PN
3,010001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for hip/knee replacement patients,Greater Than the National Average Payment,245.0,22216.0,COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Average Complications and Higher Payment,COMP,KNEE
4,010005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,Payment for heart attack patients,No Different Than the National Average Payment,71.0,24742.0,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,MORT,AMI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18786,670120,THE HOSPITALS OF PROVIDENCE TRANSMOUNTAIN CAMPUS,2000 TRANSMOUNTAIN RD,EL PASO,TX,79911,Payment for pneumonia patients,No Different Than the National Average Payment,143.0,20221.0,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Average Payment,MORT,PN
18792,670122,HOUSTON METHODIST THE WOODLANDS HOSPITAL,17201 INTERSTATE 45 SOUTH,THE WOODLANDS,TX,77385,Payment for heart attack patients,No Different Than the National Average Payment,53.0,27138.0,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,MORT,AMI
18793,670122,HOUSTON METHODIST THE WOODLANDS HOSPITAL,17201 INTERSTATE 45 SOUTH,THE WOODLANDS,TX,77385,Payment for heart failure patients,Greater Than the National Average Payment,155.0,21647.0,MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Higher Payment,MORT,HF
18794,670122,HOUSTON METHODIST THE WOODLANDS HOSPITAL,17201 INTERSTATE 45 SOUTH,THE WOODLANDS,TX,77385,Payment for pneumonia patients,No Different Than the National Average Payment,205.0,19409.0,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Better Mortality and Average Payment,MORT,PN


In [32]:
clean_value_df=clean_value_df.drop(['value_of_care_display_id'], axis=1).reset_index(drop=True)

In [33]:
clean_value_df[clean_value_df.columns[-1]] = clean_value_df[clean_value_df.columns[-1]].replace('KNEE', 'HIP_KNEE', regex=True)

In [34]:
clean_value_df['zip_code'] = clean_value_df['zip_code'].astype(str)

In [35]:
only_nj_zips=clean_value_df[clean_value_df["state"] == "NJ"]
only_nj_zips

,facility_id,facility_name,address,city,state,zip_code,payment_measure_name,payment_category,denominator,payment,value_of_care_display_name,value_of_care_category,outcome,value_code
6865,310001,HACKENSACK UNIVERSITY MEDICAL CENTER,30 PROSPECT AVE,HACKENSACK,NJ,07601,Payment for heart attack patients,Greater Than the National Average Payment,711.0,28471.0,Value of Care Heart Attack measure,Better Mortality and Higher Payment,MORT,AMI
6866,310001,HACKENSACK UNIVERSITY MEDICAL CENTER,30 PROSPECT AVE,HACKENSACK,NJ,07601,Payment for heart failure patients,Greater Than the National Average Payment,1145.0,22201.0,Value of Care Heart Failure measur,Better Mortality and Higher Payment,MORT,HF
6867,310001,HACKENSACK UNIVERSITY MEDICAL CENTER,30 PROSPECT AVE,HACKENSACK,NJ,07601,Payment for pneumonia patients,Greater Than the National Average Payment,1349.0,21324.0,Value of Care Pneumonia measure,Better Mortality and Higher Payment,MORT,PN
6868,310001,HACKENSACK UNIVERSITY MEDICAL CENTER,30 PROSPECT AVE,HACKENSACK,NJ,07601,Payment for hip/knee replacement patients,No Different Than the National Average Payment,1929.0,20870.0,Value of Care hip/knee replacement,Average Complications and Average Payment,COMP,HIP_KNEE
6869,310002,NEWARK BETH ISRAEL MEDICAL CENTER,201 LYONS AVE,NEWARK,NJ,07112,Payment for heart attack patients,No Different Than the National Average Payment,162.0,26155.0,Value of Care Heart Attack measure,Average Mortality and Average Payment,MORT,AMI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7097,310119,THE UNIVERSITY HOSPITAL,150 BERGEN ST,NEWARK,NJ,07101,Payment for pneumonia patients,No Different Than the National Average Payment,82.0,20363.0,Value of Care Pneumonia measure,Average Mortality and Average Payment,MORT,PN
7098,310130,HACKENSACK MERIDIAN HEALTH PASCACK VALLEY MEDICAL,250 OLD HOOK ROAD,WESTWOOD,NJ,07675,Payment for heart attack patients,No Different Than the National Average Payment,58.0,27250.0,Value of Care Heart Attack measure,Average Mortality and Average Payment,MORT,AMI
7099,310130,HACKENSACK MERIDIAN HEALTH PASCACK VALLEY MEDICAL,250 OLD HOOK ROAD,WESTWOOD,NJ,07675,Payment for heart failure patients,Greater Than the National Average Payment,210.0,19320.0,Value of Care Heart Failure measur,Average Mortality and Higher Payment,MORT,HF
7100,310130,HACKENSACK MERIDIAN HEALTH PASCACK VALLEY MEDICAL,250 OLD HOOK ROAD,WESTWOOD,NJ,07675,Payment for pneumonia patients,Greater Than the National Average Payment,381.0,20514.0,Value of Care Pneumonia measure,Average Mortality and Higher Payment,MORT,PN


In [36]:
clean_value_df.head(15)

,facility_id,facility_name,address,city,state,zip_code,payment_measure_name,payment_category,denominator,payment,value_of_care_display_name,value_of_care_category,outcome,value_code
0,010001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for heart attack patients,No Different Than the National Average Payment,574.0,24934.0,Value of Care Heart Attack measure,Average Mortality and Average Payment,MORT,AMI
1,010001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for heart failure patients,No Different Than the National Average Payment,823.0,17854.0,Value of Care Heart Failure measur,Average Mortality and Average Payment,MORT,HF
2,010001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for pneumonia patients,Greater Than the National Average Payment,536.0,20216.0,Value of Care Pneumonia measure,Average Mortality and Higher Payment,MORT,PN
3,010001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Payment for hip/knee replacement patients,Greater Than the National Average Payment,245.0,22216.0,Value of Care hip/knee replacement,Average Complications and Higher Payment,COMP,HIP_KNEE
4,010005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,Payment for heart attack patients,No Different Than the National Average Payment,71.0,24742.0,Value of Care Heart Attack measure,Average Mortality and Average Payment,MORT,AMI
5,010005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,Payment for heart failure patients,No Different Than the National Average Payment,271.0,16631.0,Value of Care Heart Failure measur,Worse Mortality and Average Payment,MORT,HF
6,010005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,Payment for pneumonia patients,Less Than the National Average Payment,595.0,16540.0,Value of Care Pneumonia measure,Worse Mortality and Lower Payment,MORT,PN
7,010005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,Payment for hip/knee replacement patients,No Different Than the National Average Payment,210.0,20679.0,Value of Care hip/knee replacement,Average Complications and Average Payment,COMP,HIP_KNEE
8,010006,NORTH ALABAMA MEDICAL CENTER,1701 VETERANS DRIVE,FLORENCE,AL,35630,Payment for heart attack patients,No Different Than the National Average Payment,406.0,24466.0,Value of Care Heart Attack measure,Worse Mortality and Average Payment,MORT,AMI
9,010006,NORTH ALABAMA MEDICAL CENTER,1701 VETERANS DRIVE,FLORENCE,AL,35630,Payment for heart failure patients,No Different Than the National Average Payment,625.0,17487.0,Value of Care Heart Failure measur,Average Mortality and Average Payment,MORT,HF


In [37]:
clean_value_df.groupby(['zip_code'])['value_code'].value_counts()

zip_code  value_code
00603     HF            1
          PN            1
00674     HF            2
          PN            2
          AMI           1
                       ..
99801     PN            1
99835     PN            1
99901     HF            1
          HIP_KNEE      1
          PN            1
Name: value_code, Length: 11498, dtype: int64

In [38]:
clean_value_df.to_csv("output_data/clean_value_df.csv", index=False)

### CSV on readmission rates

In [39]:
readmissions_df = pd.read_csv("data/Readmissions_Reduction_Program.csv", low_memory=False)
updated_readmissions_df=readmissions_df.drop(['Footnote', 'Start Date', 'End Date', "State", "Predicted Readmission Rate", "Expected Readmission Rate"], axis=1).reset_index(drop=True)
updated_readmissions_df.columns = updated_readmissions_df.columns.str.replace(' ', '_').str.lower()
updated_readmissions_df

,facility_name,facility_id,measure_name,number_of_discharges,excess_readmission_ratio,number_of_readmissions
0,SOUTHEAST ALABAMA MEDICAL CENTER,10001,READM-30-HIP-KNEE-HRRP,258.0,1.1735,17
1,SOUTHEAST ALABAMA MEDICAL CENTER,10001,READM-30-CABG-HRRP,268.0,1.1868,41
2,SOUTHEAST ALABAMA MEDICAL CENTER,10001,READM-30-AMI-HRRP,620.0,1.0204,100
3,SOUTHEAST ALABAMA MEDICAL CENTER,10001,READM-30-HF-HRRP,1107.0,1.0702,264
4,SOUTHEAST ALABAMA MEDICAL CENTER,10001,READM-30-COPD-HRRP,444.0,1.0560,92
...,...,...,...,...,...,...
19219,ASCENSION SETON BASTROP,670143,READM-30-CABG-HRRP,NaN,NaN,NaN
19220,ASCENSION SETON BASTROP,670143,READM-30-COPD-HRRP,NaN,NaN,NaN
19221,ASCENSION SETON BASTROP,670143,READM-30-HF-HRRP,NaN,NaN,NaN
19222,ASCENSION SETON BASTROP,670143,READM-30-HIP-KNEE-HRRP,NaN,NaN,NaN


In [40]:
no_nans_readmissions=updated_readmissions_df.dropna()
no_nans_readmissions

,facility_name,facility_id,measure_name,number_of_discharges,excess_readmission_ratio,number_of_readmissions
0,SOUTHEAST ALABAMA MEDICAL CENTER,10001,READM-30-HIP-KNEE-HRRP,258.0,1.1735,17
1,SOUTHEAST ALABAMA MEDICAL CENTER,10001,READM-30-CABG-HRRP,268.0,1.1868,41
2,SOUTHEAST ALABAMA MEDICAL CENTER,10001,READM-30-AMI-HRRP,620.0,1.0204,100
3,SOUTHEAST ALABAMA MEDICAL CENTER,10001,READM-30-HF-HRRP,1107.0,1.0702,264
4,SOUTHEAST ALABAMA MEDICAL CENTER,10001,READM-30-COPD-HRRP,444.0,1.0560,92
...,...,...,...,...,...,...
19143,THE HOSPITALS OF PROVIDENCE TRANSMOUNTAIN CAMPUS,670120,READM-30-HF-HRRP,170.0,1.1127,47
19145,THE HOSPITALS OF PROVIDENCE TRANSMOUNTAIN CAMPUS,670120,READM-30-PN-HRRP,157.0,0.9826,22
19154,HOUSTON METHODIST THE WOODLANDS HOSPITAL,670122,READM-30-COPD-HRRP,88.0,0.9997,17
19155,HOUSTON METHODIST THE WOODLANDS HOSPITAL,670122,READM-30-HF-HRRP,204.0,0.9847,43


In [41]:
no_nans_readmissions['excess_readmission_ratio'].apply(type).value_counts()

<class 'float'>    11103
Name: excess_readmission_ratio, dtype: int64

In [42]:
no_nans_readmissions['number_of_discharges'].apply(type).value_counts()

<class 'float'>    11103
Name: number_of_discharges, dtype: int64

In [43]:
no_nans_readmissions['number_of_readmissions'].apply(type).value_counts()

<class 'str'>    11103
Name: number_of_readmissions, dtype: int64

In [44]:
no_nans_readmissions['number_of_readmissions'] = no_nans_readmissions['number_of_readmissions'].apply(pd.to_numeric)

<ipython-input-44-a49fa4ce6279>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_nans_readmissions['number_of_readmissions'] = no_nans_readmissions['number_of_readmissions'].apply(pd.to_numeric)


In [45]:
no_nans_readmissions['facility_id'].apply(type).value_counts()

<class 'int'>    11103
Name: facility_id, dtype: int64

In [46]:
no_nans_readmissions.describe()

,facility_id,number_of_discharges,excess_readmission_ratio,number_of_readmissions
count,11103.000000,11103.000000,11103.000000,11103.000000
mean,257786.415293,397.165541,1.007267,63.717644
std,155964.882032,365.180104,0.088375,63.022506
min,10001.000000,26.000000,0.611200,11.000000
25%,110083.000000,161.000000,0.956200,22.000000
50%,250042.000000,292.000000,1.003900,43.000000
75%,390041.000000,509.000000,1.053600,82.000000
max,670122.000000,9285.000000,2.337800,916.000000


In [47]:
no_nans_readmissions.to_csv("output_data/readmissions_df.csv", index=False)

### Export to PostgreSQL

In [48]:
from sqlalchemy import create_engine

In [49]:
rds_connection_string = "postgres:3411@localhost:5432/project2"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [50]:
engine.table_names()

['readmissions', 'voc', 'census', 'zipcode']

In [51]:
final_census_df.to_sql(name='census', con=engine, if_exists='append', index=False)

In [52]:
clean_value_df.to_sql(name='voc', con=engine, if_exists='append', index=False)

In [53]:
no_nans_readmissions.to_sql(name='readmissions', con=engine, if_exists='append', index=False)